In [1]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import MaxPooling2D
from matplotlib import pyplot as plt
import pathlib
from tensorflow.python.ops.numpy_ops import np_config
np_config.enable_numpy_behavior()

# Discriminator

In [2]:
# define the standalone discriminator model
def define_discriminator(in_shape=(256,256,3)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    return model

# Generator

In [3]:
# define the standalone generator model
def define_generator(latent_dim):
    model = Sequential()
    # foundation for 32x32 image
    n_nodes = 128 * 64 * 64
    model.add(Dense(n_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((64, 64, 128)))
    # upsample to 128x128
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    # upsample to 256x256
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same', activation='tanh'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(3, (8,8), activation='sigmoid', padding='same'))
    return model

# GAN

In [4]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
    # make weights in the discriminator not trainable
    d_model.trainable = False
    # connect them
    model = Sequential()
    # add generator
    model.add(g_model)
    # add the discriminator
    model.add(d_model)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt)
    return model

# Support Functions

In [5]:
def process_path(file_path):
    image = tf.io.read_file(file_path)
    image = tf.io.decode_jpeg(image,channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [256,256])
    image = tf.reshape(image, [256,256,3])
    return image

# load and prepare monet training images
def load_real_samples_monet():
    monet_images = pathlib.Path('/home/sean/.keras/datasets/monet_jpg')
    monet_images_list_ds = tf.data.Dataset.list_files(str(monet_images/'*'))
    monet_ds = []
    for i in monet_images_list_ds:
        monet_ds.append(process_path(i))

    trainX = tf.convert_to_tensor(monet_ds)
    plot_samples(trainX, title='Monet Images\n')
    # print(trainX.shape)
    # expand to 3d, e.g. add channels dimension
    X = expand_dims(trainX, axis=-1)
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X

# load and prepare photo training images
def load_real_samples_photo():
    photo_images = pathlib.Path('/home/sean/.keras/datasets/photo_jpg_cut')
    photo_images_list_ds = tf.data.Dataset.list_files(str(photo_images/'*'))
    photo_ds = []
    for i in photo_images_list_ds:
        photo_ds.append(process_path(i))

    trainX = tf.convert_to_tensor(photo_ds)
    plot_samples(trainX, title='Photo Images\n')
    # expand to 3d, e.g. add channels dimension
    X = expand_dims(trainX, axis=-1)
    # convert from unsigned ints to floats
    X = X.astype('float32')
    # scale from [0,255] to [0,1]
    X = X / 255.0
    return X
 
# select real samples
def generate_real_samples(dataset, n_samples):
    # choose random instances
    ix = randint(0, dataset.shape[0], n_samples)
    # retrieve selected images
    X = dataset[ix]
    # generate 'real' class labels (1)
    y = ones((n_samples, 1))
    return X, y
 
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input
 
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
    # generate points in latent space
    x_input = generate_latent_points(latent_dim, n_samples)
    # predict outputs
    X = g_model.predict(x_input)
    # print('fake samples:,',X.shape)
    # create 'fake' class labels (0)
    y = zeros((n_samples, 1))
    return X, y
 
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
    # plot images
    for i in range(n * n):
        # define subplot
        plt.subplot(n, n, 1 + i)
        # turn off axis
        plt.axis('off')
        # plot raw pixel data
        plt.imshow(examples[i, :, :, 0], cmap='gray_r')
    # save plot to file
    filename = 'generated_plot_e%03d.png' % (epoch+1)
    plt.savefig(filename)
    plt.close()

def plot_samples(input_array, rows=4, cols=5, title=''):
    '''
    Function to plot 256x256 pixel drawings that are stored in a numpy array.
    Specify how many rows and cols of pictures to display (default 4x5).  
    If the array contains less images than subplots selected, surplus subplots remain empty.
    '''    
    fig, ax = plt.subplots(figsize=(cols,rows))
    ax.axis('off')
    plt.title(title)

    for i in list(range(0, min(len(input_array),(rows*cols)) )):      
        a = fig.add_subplot(rows,cols,i+1)
        imgplot = plt.imshow(input_array[i,:784].reshape((256,256,3)), cmap='gray_r', interpolation='nearest')
        plt.xticks([])
        plt.yticks([])
        
    plt.show()
 
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
    # prepare real samples
    X_real, y_real = generate_real_samples(dataset, n_samples)
    # evaluate discriminator on real examples
    _, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
    # prepare fake examples
    x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
    # evaluate discriminator on fake examples
    _, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
    # summarize discriminator performance
    print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
    plot_samples(x_fake, title='Monet Images\n')

# Training

In [6]:
# train the generator and discriminator
def train(g_modelA, d_modelA, gan_modelA, datasetA, g_modelB, d_modelB, gan_modelB, datasetB, latent_dim, n_epochs=100, n_batch=15):
    bat_per_epo = int(datasetA.shape[0] / n_batch)
    half_batch = int(n_batch / 2)
    # print(dataset.shape[0])
    # print(n_batch)
    # manually enumerate epochs
    for i in range(n_epochs):
        # enumerate batches over the training set
        for j in range(bat_per_epo):
            # get randomly selected 'real' samples from A
            X_realA, y_realA = generate_real_samples(datasetA, half_batch)
            # get randomly selected 'real' samples from B
            X_realB, y_realB = generate_real_samples(datasetB, half_batch)
            
            # generate 'fake' examples for A
            X_fakeA, y_fakeA = generate_fake_samples(g_modelA, latent_dim, half_batch)
            # generate 'fake' examples for B
            X_fakeB, y_fakeB = generate_fake_samples(g_modelB, latent_dim, half_batch)
            
            # create training set for the discriminator A
            X_realA = X_realA.reshape(X_fakeA.shape)
            X_fakeA = X_fakeA.reshape(X_fakeA.shape)
            # create training set for the discriminator B
            X_realB = X_realB.reshape(X_fakeB.shape)
            X_fakeB = X_fakeB.reshape(X_fakeB.shape)
            
            XA, yA = vstack((X_realA, X_fakeA)), vstack((y_realA, y_fakeA))
            XB, yB = vstack((X_realB, X_fakeB)), vstack((y_realB, y_fakeB))
            
            
            # update discriminator model weights for A
            d_lossA = d_modelA.train_on_batch(XA, yA)
            # update discriminator model weights for B
            d_lossB = d_modelB.train_on_batch(XB, yB)
                                                        
            # prepare points in latent space as input for the generator for A
            X_ganA = generate_latent_points(latent_dim, n_batch)
            # prepare points in latent space as input for the generator for B
            X_ganB = generate_latent_points(latent_dim, n_batch)
                                                        
            # create inverted labels for the fake samples for A                                    
            y_ganA = ones((n_batch, 1))
            # create inverted labels for the fake samples for B                                 
            y_ganB = ones((n_batch, 1))
                                                        
            # update the generator via the discriminator's error for A
            g_lossA = gan_modelA.train_on_batch(X_ganA, y_ganA)
            # update the generator via the discriminator's error for B
            g_lossB = gan_modelB.train_on_batch(X_ganB, y_ganB)
                                                        
            # summarize loss on this batch for A
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_lossA, g_lossA))
            # summarize loss on this batch for B
            print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_lossB, g_lossB))
                                                        
        # evaluate the model performance, sometimes
        if (i+1) % 1 == 0:
            summarize_performance(i, g_modelA, d_modelA, datasetA, latent_dim)
            summarize_performance(i, g_modelB, d_modelB, datasetB, latent_dim)

In [ ]:
# size of the latent space
latent_dim = 100

# create the monet discriminator
d_model_monet = define_discriminator()
print('MONET DISCRIMINATOR')
d_model_monet.summary()
# create the monet generator
g_model_monet = define_generator(latent_dim)
print('MONET GENERATOR')
g_model_monet.summary()


# create the photo discriminator
d_model_photo = define_discriminator()
print('PHOTO DISCRIMINATOR')
d_model_photo.summary()
# create the photo generator
g_model_photo = define_generator(latent_dim)
print('PHOTO GENERATOR')
g_model_photo.summary()

# create the monet gan
gan_model_monet = define_gan(g_model_monet, d_model_photo)
print('MONET GAN')
gan_model_monet.summary()
# create the photo gan
gan_model_photo = define_gan(g_model_photo, d_model_monet)
print('PHOTO GAN')
gan_model_photo.summary()

# load image data
monet_dataset = load_real_samples_monet()
photo_dataset = load_real_samples_photo()

# train model
train(g_model_monet, d_model_monet, gan_model_monet, monet_dataset, g_model_photo, d_model_photo, gan_model_photo, photo_dataset, latent_dim)

MONET DISCRIMINATOR
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 leaky_re_lu (LeakyReLU)     (None, 128, 128, 64)      0         
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 64)        36928     
                                                                 
 leaky_re_lu_1 (LeakyReLU)   (None, 64, 64, 64)        0         
                                                                 
 dropout_1 (Dropout)         (None, 64, 64, 64)        0         
                                                                 
 flatten (Flatten)           (None, 

/home/sean/.local/lib/python3.8/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


MONET GENERATOR
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 524288)            52953088  
                                                                 
 leaky_re_lu_2 (LeakyReLU)   (None, 524288)            0         
                                                                 
 reshape (Reshape)           (None, 64, 64, 128)       0         
                                                                 
 conv2d_transpose (Conv2DTra  (None, 128, 128, 128)    262272    
 nspose)                                                         
                                                                 
 leaky_re_lu_3 (LeakyReLU)   (None, 128, 128, 128)     0         
                                                                 
 conv2d_transpose_1 (Conv2DT  (None, 256, 256, 128)    262272    
 ranspose)                            